## Beam Energy Prediction with beam energy as a categorical variable and particle name as dummies

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from IPython.display import Image

In [8]:
df = pd.read_csv("beam_energies_H_or_L.csv")

In [9]:
df

,Unnamed: 0,Layer1,Layer2,Layer3,Layer4,Layer5,Layer6,Layer7,Layer8,Layer9,...,Layer33,Layer34,Layer35,Layer36,Layer37,Layer38,Layer39,Layer40,Beam Energy,Particle
0,0,77.66750,259.46000,792.90200,1239.53000,1980.100000,2386.750000,3089.27000,2696.600000,3145.130000,...,13.75550,9.54401,11.20690,7.76882,4.04211,1.02878,0.00000,1.443850,H,ele
1,1,38.42920,104.51000,371.66200,603.23100,1079.270000,1288.220000,1780.75000,2124.150000,2654.140000,...,11.72670,5.71907,6.14952,3.77939,9.07730,1.57805,0.00000,0.673109,H,ele
2,2,45.87910,84.95750,234.06200,326.84000,747.987000,1189.420000,1276.22000,1621.180000,2513.580000,...,12.90330,6.34586,10.02230,5.76236,3.23882,1.32135,0.00000,0.000000,H,ele
3,3,430.59100,361.49100,838.59600,1177.32000,1897.690000,2585.080000,2679.29000,2515.590000,2648.980000,...,6.56424,2.21363,12.74250,5.27991,5.18920,0.00000,0.00000,0.000000,H,ele
4,4,161.45900,32.52370,215.43900,474.61400,1038.740000,1268.180000,1672.78000,1824.650000,2239.280000,...,10.62380,6.06490,13.94180,7.57974,3.18496,0.00000,0.00000,0.000000,H,ele
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,1.19195,8.26787,1.44395,2.55059,1.397010,1.168870,1.31133,2.762510,1.478290,...,13.02580,6.40018,5.32122,12.80920,6.69577,1.37853,1.49050,1.003560,H,muon
496,496,1.18656,2.07798,2.14504,1.04400,1.079860,0.801306,1.30868,1.594340,1.210860,...,16.35610,14.24660,11.63200,11.69070,4.52460,1.06547,6.94639,2.387170,H,muon
497,497,1.05722,1.53828,1.05049,1.53319,3.637990,0.989049,1.75165,1.084330,0.978782,...,15.42560,7.69472,14.90430,22.06300,8.79396,2.04097,1.23077,0.788855,H,muon
498,498,1.02478,2.87172,1.17401,1.45174,0.977608,1.571210,1.27423,0.778268,1.143040,...,9.78310,8.41551,6.44172,5.41960,6.38096,1.53922,0.00000,0.472999,H,muon


In [12]:
#One-hot encoder for turning particle names into dummies because model does not handle categorical variables. 
def one_hot_encoder_for_particle(df):
    original_columns = list(df.columns)
    categorical_column = ["Particle"]
    df = pd.get_dummies(df, columns= categorical_column)
    return df

In [13]:
df = one_hot_encoder_for_particle(df)

In [18]:
df.columns

Index(['Unnamed: 0', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5',
       'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10', 'Layer11', 'Layer12',
       'Layer13', 'Layer14', 'Layer15', 'Layer16', 'Layer17', 'Layer18',
       'Layer19', 'Layer20', 'Layer21', 'Layer22', 'Layer23', 'Layer24',
       'Layer25', 'Layer26', 'Layer27', 'Layer28', 'Layer29', 'Layer30',
       'Layer31', 'Layer32', 'Layer33', 'Layer34', 'Layer35', 'Layer36',
       'Layer37', 'Layer38', 'Layer39', 'Layer40', 'Beam Energy',
       'Particle_ele', 'Particle_muon', 'Particle_pion'],
      dtype='object')

In [19]:
df.drop(labels="Unnamed: 0", axis =1, inplace= True)

In [20]:
df

,Layer1,Layer2,Layer3,Layer4,Layer5,Layer6,Layer7,Layer8,Layer9,Layer10,...,Layer35,Layer36,Layer37,Layer38,Layer39,Layer40,Beam Energy,Particle_ele,Particle_muon,Particle_pion
0,77.66750,259.46000,792.90200,1239.53000,1980.100000,2386.750000,3089.27000,2696.600000,3145.130000,2149.210000,...,11.20690,7.76882,4.04211,1.02878,0.00000,1.443850,H,1,0,0
1,38.42920,104.51000,371.66200,603.23100,1079.270000,1288.220000,1780.75000,2124.150000,2654.140000,2243.690000,...,6.14952,3.77939,9.07730,1.57805,0.00000,0.673109,H,1,0,0
2,45.87910,84.95750,234.06200,326.84000,747.987000,1189.420000,1276.22000,1621.180000,2513.580000,2167.400000,...,10.02230,5.76236,3.23882,1.32135,0.00000,0.000000,H,1,0,0
3,430.59100,361.49100,838.59600,1177.32000,1897.690000,2585.080000,2679.29000,2515.590000,2648.980000,2107.160000,...,12.74250,5.27991,5.18920,0.00000,0.00000,0.000000,H,1,0,0
4,161.45900,32.52370,215.43900,474.61400,1038.740000,1268.180000,1672.78000,1824.650000,2239.280000,1999.550000,...,13.94180,7.57974,3.18496,0.00000,0.00000,0.000000,H,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1.19195,8.26787,1.44395,2.55059,1.397010,1.168870,1.31133,2.762510,1.478290,1.073920,...,5.32122,12.80920,6.69577,1.37853,1.49050,1.003560,H,0,1,0
496,1.18656,2.07798,2.14504,1.04400,1.079860,0.801306,1.30868,1.594340,1.210860,2.192870,...,11.63200,11.69070,4.52460,1.06547,6.94639,2.387170,H,0,1,0
497,1.05722,1.53828,1.05049,1.53319,3.637990,0.989049,1.75165,1.084330,0.978782,0.868786,...,14.90430,22.06300,8.79396,2.04097,1.23077,0.788855,H,0,1,0
498,1.02478,2.87172,1.17401,1.45174,0.977608,1.571210,1.27423,0.778268,1.143040,0.929430,...,6.44172,5.41960,6.38096,1.53922,0.00000,0.472999,H,0,1,0


In [22]:
#to predict the beam energies
def split_dataset(df): 
    # Separating the target variable
    X = df.drop("Beam Energy", inplace=False, axis=1)
    Y = df["Beam Energy"]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                        test_size = 0.2, 
                                                        random_state = 100, 
                                                        shuffle = True)
    return X, Y, X_train, X_test, y_train, y_test

In [23]:
def train_using_gini(X_train, X_test, y_train):
  
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion = "gini",
                                      random_state = 100,
                                      max_depth=3, min_samples_leaf=5)
  
    # Performing training
    clf_gini.fit(X_train, y_train)
    return clf_gini

In [24]:
def train_using_entropy(X_train, X_test, y_train):
  
    # Decision tree with entropy
    clf_entropy = DecisionTreeClassifier(criterion = "entropy",
                                         random_state = 100,
                                         max_depth = 3,
                                         min_samples_leaf = 5)
  
    # Performing training
    clf_entropy.fit(X_train, y_train)
    return clf_entropy

In [25]:
def prediction(X_test, clf_object):
  
    # Predicton on test with giniIndex
    y_pred = clf_object.predict(X_test)
    indices = list(X_test.index)
    data = {"Index": indices, "Prediction": y_pred}
    y_pred_df = pd.DataFrame(data = data)
    print(y_pred_df)
    y_pred_df.to_csv("Predictions.csv", index=False)
    return y_pred

In [26]:
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: \n",
        confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",
    accuracy_score(y_test,y_pred)*100)
      
    print("Report : ",
    classification_report(y_test, y_pred))

In [27]:
def exp1():
      
    # Building Phase
    data = df
    X, Y, X_train, X_test, y_train, y_test = split_dataset(data)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = train_using_entropy(X_train, X_test, y_train)
    print("Actual results:")
    print(y_test)
    # Operational Phase
    print()
    print("Predictions Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini) 
    cal_accuracy(y_test, y_pred_gini)
    print()
    print("Predictions Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
      

In [28]:
exp1()

Actual results:
69     H
29     H
471    H
344    L
54     H
      ..
460    H
152    L
154    L
56     H
392    L
Name: Beam Energy, Length: 100, dtype: object

Predictions Using Gini Index:
    Index Prediction
0      69          H
1      29          H
2     471          H
3     344          L
4      54          H
..    ...        ...
95    460          H
96    152          L
97    154          L
98     56          H
99    392          H

[100 rows x 2 columns]
Confusion Matrix: 
 [[56  0]
 [ 3 41]]
Accuracy :  97.0
Report :                precision    recall  f1-score   support

           H       0.95      1.00      0.97        56
           L       1.00      0.93      0.96        44

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100


Predictions Using Entropy:
    Index Prediction
0      69          H
1      29          H
2     471          H
3     344          L
4     

Our model predicted the beam energies as H or L with a 97% accuracy, which is quite good. 
